In [ ]:
!pip install --use-deprecated=legacy-resolver git+https://github.com/pycaret/pycaret.git

In [ ]:
!pip install --use-deprecated=legacy-resolver pycaret[full]

In [ ]:
!pip install statsmodels
!pip install pystan~=2.14
!pip install fbprophet
!pip install jupyter-dash
!pip install ruptures

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet
import ruptures as rpt
from sklearn.metrics import r2_score

## dash imports
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from pycaret.regression import *


In [ ]:
df = pd.read_parquet("2020-2022_history_data_processed.parquet.brotli")
df.head()

,Station,City,State,Date,Time,PM25,PM10,CO,NO2,NH3,O3,SO2,AQI,AQI_bucket
0,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,10:00:00,32.76,37.19,0.57,6.28,11.40,130.18,12.88,176.0,Moderate
1,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,11:00:00,43.00,50.81,0.82,13.66,19.00,97.28,15.50,176.0,Moderate
2,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,12:00:00,51.04,61.88,1.04,22.77,25.08,63.66,17.17,176.0,Moderate
3,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,13:00:00,55.24,68.78,1.12,26.22,29.64,42.20,16.69,176.0,Moderate
4,"Adarsh Nagar, Jaipur, India",Jaipur,Rajasthan,2020-11-28,14:00:00,58.45,73.32,1.12,24.04,29.89,33.98,15.26,176.0,Moderate


In [ ]:
data = df.groupby(['Station','Date']).mean().reset_index()
data.head()

,Station,Date,PM25,PM10,CO,NO2,NH3,O3,SO2,AQI
0,"Adarsh Nagar, Jaipur, India",2020-11-28,57.477143,68.166429,0.923571,16.052857,24.391429,51.218571,11.765000,147.357143
1,"Adarsh Nagar, Jaipur, India",2020-11-29,103.115833,117.580417,1.337500,20.146250,23.140000,73.754167,18.125000,214.333333
2,"Adarsh Nagar, Jaipur, India",2020-11-30,115.448333,135.011667,1.555833,28.050833,41.944167,59.497083,28.493750,273.375000
3,"Adarsh Nagar, Jaipur, India",2020-12-01,100.867917,115.581667,1.210000,19.860000,27.761250,73.208333,22.220000,270.708333
4,"Adarsh Nagar, Jaipur, India",2020-12-02,116.007083,132.211250,1.280417,22.424167,29.892917,68.062917,22.336667,277.666667


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data['month'] = [i.month for i in data['Date']]
data['year'] = [i.year for i in data['Date']]
data['day_of_week'] = [i.dayofweek for i in data['Date']]
data['day_of_year'] = [i.dayofyear for i in data['Date']]

data.columns

Index(['Station', 'Date', 'PM25', 'PM10', 'CO', 'NO2', 'NH3', 'O3', 'SO2',
       'AQI', 'month', 'year', 'day_of_week', 'day_of_year'],
      dtype='object')

In [ ]:
stations = data['Station'].unique().tolist()

data=data.drop([ 'PM25', 'PM10', 'CO', 'NO2', 'NH3', 'O3', 'SO2'],axis=1)
data.set_index('Date',inplace=True)
data.head()

,Station,AQI,month,year,day_of_week,day_of_year
Date,,,,,,
2020-11-28,"Adarsh Nagar, Jaipur, India",147.357143,11,2020,5,333
2020-11-29,"Adarsh Nagar, Jaipur, India",214.333333,11,2020,6,334
2020-11-30,"Adarsh Nagar, Jaipur, India",273.375000,11,2020,0,335
2020-12-01,"Adarsh Nagar, Jaipur, India",270.708333,12,2020,1,336
2020-12-02,"Adarsh Nagar, Jaipur, India",277.666667,12,2020,2,337


In [ ]:
dates = pd.date_range(start='2020-11-28', end = '2022-02-09', freq = 'D')
ma_df = pd.DataFrame()
ma_df['Date'] = dates
ma_df.set_index('Date',inplace=True)
ma_df.head()

""
Date
2020-11-28
2020-11-29
2020-11-30
2020-12-01
2020-12-02


In [ ]:
# plot multiple time series with moving avgs in a loop

for i in stations:
  subset = data[data['Station'] == i]
  subset[f'{i}_MA'] = subset['AQI'].rolling(30).mean()
  subset.rename(columns = {'AQI': f'{i}_AQI'}, inplace = True)
  subset.drop(columns=['Station','month','year','day_of_week','day_of_year'],inplace=True)
  ma_df = pd.merge(ma_df, subset, left_index=True, right_index=True)
ma_df.head()


,"Adarsh Nagar, Jaipur, India_AQI","Adarsh Nagar, Jaipur, India_MA","Alandur Bus Depot, Chennai, Chennai, India_AQI","Alandur Bus Depot, Chennai, Chennai, India_MA","Anand Kala Kshetram, Rajamahendravaram, India_AQI","Anand Kala Kshetram, Rajamahendravaram, India_MA","Anand Vihar, Delhi, Delhi, India_AQI","Anand Vihar, Delhi, Delhi, India_MA","Anand Vihar, Hapur, India_AQI","Anand Vihar, Hapur, India_MA","Anthoni Pillai Nagar, Gummidipoondi, India_AQI","Anthoni Pillai Nagar, Gummidipoondi, India_MA","Arya Nagar, Bahadurgarh, India_AQI","Arya Nagar, Bahadurgarh, India_MA","Asansol Court Area, Asansol, India_AQI","Asansol Court Area, Asansol, India_MA","Ashok Nagar, Udaipur, India_AQI","Ashok Nagar, Udaipur, India_MA","B.Katihalli, Hassan, India_AQI","B.Katihalli, Hassan, India_MA","BWSSB Kadabesanahalli, Bengaluru, India_AQI","BWSSB Kadabesanahalli, Bengaluru, India_MA","Bandhavgar Colony, Satna, India_AQI","Bandhavgar Colony, Satna, India_MA","Bhopal Chauraha, Dewas, India_AQI","Bhopal Chauraha, Dewas, India_MA","Borivali East MPCB, Mumbai, India_AQI","Borivali East MPCB, Mumbai, India_MA","Borivali East, Mumbai, India_AQI","Borivali East, Mumbai, India_MA","Central Jail, Buxar, India_AQI","Central Jail, Buxar, India_MA","Chhoti Gwaltoli, Indore, India_AQI","Chhoti Gwaltoli, Indore, India_MA","Chikkaballapur Rural, Chikkaballapur, India_AQI","Chikkaballapur Rural, Chikkaballapur, India_MA","City Center, Gwalior, India_AQI","City Center, Gwalior, India_MA","Civil Line, Jalandhar, India_AQI","Civil Line, Jalandhar, India_MA","Civil Lines, Ajmer, India_AQI","Civil Lines, Ajmer, India_MA","Collector Office, Yadgir, India_AQI","Collector Office, Yadgir, India_MA","Collectorate, Jodhpur, India_AQI","Collectorate, Jodhpur, India_MA","Corporation Ground, Thrissur, India_AQI","Corporation Ground, Thrissur, India_MA","D M Colony, Bihar Sharif, India_AQI","D M Colony, Bihar Sharif, India_MA","Dada Peer, Sasaram, India_AQI","Dada Peer, Sasaram, India_MA","Dangi Tola, Rajgir, India_AQI","Dangi Tola, Rajgir, India_MA","Darshan Nagar, Chhapra, India_AQI","Darshan Nagar, Chhapra, India_MA","Deen Dayal Nagar, Sagar, India_AQI","Deen Dayal Nagar, Sagar, India_MA","Deonar, Mumbai, India_AQI","Deonar, Mumbai, India_MA","Deshpande Nagar, Hubballi, India_AQI","Deshpande Nagar, Hubballi, India_MA","Devaraj Urs Badavane, Davanagere, India_AQI","Devaraj Urs Badavane, Davanagere, India_MA","Diwator Nagar, Koppal, India_AQI","Diwator Nagar, Koppal, India_MA","F-Block, Sirsa, India_AQI","F-Block, Sirsa, India_MA","FTI Kidwai Nagar, Kanpur, India_AQI","FTI Kidwai Nagar, Kanpur, India_MA","GIDC, Ankleshwar, India_AQI","GIDC, Ankleshwar, India_MA","GIDC, Nandesari, India_AQI","GIDC, Nandesari, India_MA","GM Office, Brajrajnagar, India_AQI","GM Office, Brajrajnagar, India_MA","GVM Corporation, Visakhapatnam, India_AQI","GVM Corporation, Visakhapatnam, India_MA","Gandak Colony, Motihari, India_AQI","Gandak Colony, Motihari, India_MA","Ganga Nagar, Meerut, India_AQI","Ganga Nagar, Meerut, India_MA","Gangapur Road, Nashik, India_AQI","Gangapur Road, Nashik, India_MA","General Hospital, Mandikhera, India_AQI","General Hospital, Mandikhera, India_MA","Ghusuri, Howrah, India_AQI","Ghusuri, Howrah, India_MA","Golden Temple, Amritsar, India_AQI","Golden Temple, Amritsar, India_MA","Gole Bazar, Katni, India_AQI","Gole Bazar, Katni, India_MA","H.B. Colony, Bhiwani, India_AQI","H.B. Colony, Bhiwani, India_MA","Haji Colony, Raichur, India_AQI","Haji Colony, Raichur, India_MA","Haldia, Haldia, India_AQI","Haldia, Haldia, India_MA","Hardev Nagar, Bathinda, India_AQI","Hardev Nagar, Bathinda, India_MA","Hebbal 1st Stage, Mysuru, India_AQI","Hebbal 1st Stage, Mysuru, India_MA","Huda Sector, Fatehabad, India_AQI","Huda Sector, Fatehabad, India_MA","Hyderabad US Consulate, India (हैदराबाद अमेरिकी वाणिज्य दूतावास)_AQI","Hyderabad US Consulate, India (हैदराबाद अमेरिकी वाणिज्य दूतावास)_MA","ICRISAT Patancheru, Hyderabad, India_AQI","ICRISAT Patancheru, Hyderabad, India_MA","IGSC

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} 
                 for x in stations],
        value="Adarsh Nagar, Jaipur, India",
        placeholder="Select a station"
    ),
    dcc.Graph(id="line-chart")
])

@app.callback(
    Output("line-chart", "figure"), 
    Input("dropdown", "value"))
def update_line_chart(station):

    station_df = ma_df[[f'{station}_AQI',f'{station}_MA']]

    fig = px.line(data_frame=station_df,
        x=station_df.index, y=f'{station}_AQI', title = f'{station} Moving average and AQI', template = 'plotly_dark')  # 'plotly_dark'

    fig.add_trace(go.Scatter( x=station_df.index, y=station_df[f'{station}_MA'].to_numpy(), name="Moving Avg", line_shape='linear'))
    return fig

app.run_server(mode='inline')

<IPython.core.display.Javascript object>

### Model Training

In [ ]:
all_ts = data['Station'].unique()
all_results = []
final_model = {}

for i in all_ts:
    df_subset = data[data['Station'] == i]
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'AQI', train_size = 0.95, transform_target = True, remove_outliers = True, data_split_shuffle = False,
              fold_strategy = 'timeseries', fold = 5, ignore_features = ['Station'], numeric_features = ['day_of_year', 'year'],
              categorical_features = ['month', 'day_of_week'], silent = True, verbose = False, session_id = 2022)
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=True)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['Station'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name='trained_station_models/' + str(i), verbose=False)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,75.7342,11512.3638,95.9745,-0.8068,0.6481,0.7347,0.080
lightgbm,Light Gradient Boosting Machine,76.3953,11376.3286,97.7547,-0.8840,0.6993,0.7766,0.170
catboost,CatBoost Regressor,90.6553,12323.8029,109.4173,-8.4495,0.8202,1.0921,0.650
gbr,Gradient Boosting Regressor,105.1015,16891.3617,128.7933,-17.6893,0.9122,1.4039,0.062
ada,AdaBoost Regressor,106.0093,17274.5496,130.0781,-17.8242,0.9348,1.4553,0.076
br,Bayesian Ridge,107.7337,16883.7959,127.8699,-17.9774,0.9047,1.3861,0.024
ridge,Ridge Regression,110.2183,17815.8693,131.4970,-18.0785,0.9219,1.3942,0.022
huber,Huber Regressor,110.9090,18001.3552,131.4619,-23.8810,0.9273,1.5609,0.048
rf,Random Forest Regressor,114.3993,19832.3103,137.8759,-26.6357,0.9413,1.5900,0.258
lr,Linear Regression,118.5396,20683.0570,139.8983,-25.7905,0.9352,1.6370,0.022


In [ ]:
concat_results = pd.concat(all_results,axis=0)

In [ ]:
concat_results.to_csv('best_station_models.csv',index=False)

In [ ]:
concat_results = pd.read_csv('best_station_models.csv')
concat_results.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),Station
0,K Neighbors Regressor,57.4798,5676.9787,72.0921,-0.8949,0.4440,0.4062,0.080,"Adarsh Nagar, Jaipur, India"
1,Light Gradient Boosting Machine,51.8714,5426.8815,68.5124,-0.1231,0.5071,0.4278,0.172,"Alandur Bus Depot, Chennai, Chennai, India"
2,Decision Tree Regressor,62.3537,8602.8082,80.4948,-0.6087,0.5660,0.6115,0.022,"Anand Kala Kshetram, Rajamahendravaram, India"
3,Gradient Boosting Regressor,115.2948,23836.2743,147.4204,-0.3182,0.4750,0.4378,0.062,"Anand Vihar, Delhi, Delhi, India"
4,Gradient Boosting Regressor,87.4975,11264.8268,103.9046,-0.5637,0.4704,0.4990,0.060,"Anand Vihar, Hapur, India"


### Forecasting

In [ ]:

all_dates = pd.date_range(start='2022-02-10', end = '2022-02-28', freq = 'D')
# create empty dataframe
score_df = pd.DataFrame()
# add columns to dataset
score_df['Date'] = all_dates
score_df['month'] = [i.month for i in score_df['Date']]
score_df['year'] = [i.year for i in score_df['Date']]
score_df['day_of_week'] = [i.dayofweek for i in score_df['Date']]
score_df['day_of_year'] = [i.dayofyear for i in score_df['Date']]
score_df.head()

,Date,month,year,day_of_week,day_of_year
0,2022-02-10,2,2022,3,41
1,2022-02-11,2,2022,4,42
2,2022-02-12,2,2022,5,43
3,2022-02-13,2,2022,6,44
4,2022-02-14,2,2022,0,45


In [ ]:
final_forecast = score_df.copy()
final_forecast.drop(columns=['month','year','day_of_week','day_of_year'],inplace=True)
final_forecast.head()

,Date
0,2022-02-10
1,2022-02-11
2,2022-02-12
3,2022-02-13
4,2022-02-14


In [ ]:
for i in stations:
    l = load_model('/content/drive/MyDrive/AQI/trained_station_models/' + str(i), verbose=False)
    p = predict_model(l, data=score_df)
    p.rename(columns = {'Label': f'{i}_Label'}, inplace = True)
    p.drop(columns=['month','year','day_of_week','day_of_year'],inplace=True)
    final_forecast = pd.merge(final_forecast, p, how = 'left', left_on='Date', right_on ='Date')

In [ ]:
final_forecast.head()

,Date,"Adarsh Nagar, Jaipur, India_Label","Alandur Bus Depot, Chennai, Chennai, India_Label","Anand Kala Kshetram, Rajamahendravaram, India_Label","Anand Vihar, Delhi, Delhi, India_Label","Anand Vihar, Hapur, India_Label","Anthoni Pillai Nagar, Gummidipoondi, India_Label","Arya Nagar, Bahadurgarh, India_Label","Asansol Court Area, Asansol, India_Label","Ashok Nagar, Udaipur, India_Label",...,"Urban Estate-II, Hisar, India_Label","Urban, Chamarajanagar, India_Label","Vidayagiri, Bagalkot, India_Label","Vijay Nagar, Ramanagara, India_Label","Vikas Sadan Gurgaon, Gurgaon, India_Label","Vinoba Nagara, Shivamogga, India_Label","Vyttila, Kochi, India_Label","Ward-32 Bapupara, Siliguri, India_Label","Yamunapuram, Bulandshahr, India_Label","Zoo Park, Bahadurpura West, Hyderabad, India_Label"
0,2022-02-10,262.040488,149.051495,146.052628,593.331800,446.322695,103.010796,597.978859,316.569300,122.308792,...,312.694478,110.766436,182.953136,134.987317,468.759916,120.842102,157.277519,197.725803,446.320754,277.156641
1,2022-02-11,275.976208,155.957673,146.052628,636.027969,429.657450,106.838914,578.203234,318.955894,129.405628,...,289.135091,110.757357,173.435384,138.702242,394.610132,120.842102,163.996636,191.674234,443.115248,296.653112
2,2022-02-12,320.089208,122.596903,146.052628,606.417263,395.526853,100.276683,576.910828,329.480784,124.962100,...,319.583542,109.811080,164.750150,130.291088,447.231604,120.842102,159.070013,198.042351,466.424051,310.594812
3,2022-02-13,340.663902,108.310548,195.125000,680.846293,497.634545,93.114113,581.210369,327.153470,128.400943,...,349.376640,107.560620,161.887375,126.748637,445.676675,120.842102,150.996117,208.186746,513.350217,276.325122
4,2022-02-14,349.984984,120.673307,209.000000,670.568310,502.749126,87.506212,586.739184,320.583670,126.009068,...,355.788525,109.854095,163.012333,128.289254,470.524936,120.842102,162.765745,209.620314,521.557742,256.954772


In [ ]:
final_forecast.to_csv("final_station_forecast.csv",index=False)

In [ ]:
final_forecast = pd.read_csv('final_station_forecast.csv')
final_forecast.head()

,Date,"Adarsh Nagar, Jaipur, India_Label","Alandur Bus Depot, Chennai, Chennai, India_Label","Anand Kala Kshetram, Rajamahendravaram, India_Label","Anand Vihar, Delhi, Delhi, India_Label","Anand Vihar, Hapur, India_Label","Anthoni Pillai Nagar, Gummidipoondi, India_Label","Arya Nagar, Bahadurgarh, India_Label","Asansol Court Area, Asansol, India_Label","Ashok Nagar, Udaipur, India_Label",...,"Urban Estate-II, Hisar, India_Label","Urban, Chamarajanagar, India_Label","Vidayagiri, Bagalkot, India_Label","Vijay Nagar, Ramanagara, India_Label","Vikas Sadan Gurgaon, Gurgaon, India_Label","Vinoba Nagara, Shivamogga, India_Label","Vyttila, Kochi, India_Label","Ward-32 Bapupara, Siliguri, India_Label","Yamunapuram, Bulandshahr, India_Label","Zoo Park, Bahadurpura West, Hyderabad, India_Label"
0,2022-02-10,262.040488,149.051495,146.052628,593.331800,446.322695,103.010796,597.978859,316.569300,122.308792,...,312.694478,110.766436,182.953136,134.987317,468.759916,120.842102,157.277519,197.725803,446.320754,277.156641
1,2022-02-11,275.976208,155.957673,146.052628,636.027969,429.657450,106.838914,578.203234,318.955894,129.405628,...,289.135091,110.757357,173.435384,138.702242,394.610132,120.842102,163.996636,191.674234,443.115248,296.653112
2,2022-02-12,320.089208,122.596903,146.052628,606.417263,395.526853,100.276683,576.910828,329.480784,124.962100,...,319.583542,109.811080,164.750150,130.291088,447.231604,120.842102,159.070013,198.042351,466.424051,310.594812
3,2022-02-13,340.663902,108.310548,195.125000,680.846293,497.634545,93.114113,581.210369,327.153470,128.400943,...,349.376640,107.560620,161.887375,126.748637,445.676675,120.842102,150.996117,208.186746,513.350217,276.325122
4,2022-02-14,349.984984,120.673307,209.000000,670.568310,502.749126,87.506212,586.739184,320.583670,126.009068,...,355.788525,109.854095,163.012333,128.289254,470.524936,120.842102,162.765745,209.620314,521.557742,256.954772


In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} for x in stations],
        value="Adarsh Nagar, Jaipur, India",
        placeholder="Select a station"
    ),
    dcc.Graph(id="line-chart")
])

@app.callback(
    Output("line-chart", "figure"), 
    Input("dropdown", "value"))
def update_line_chart(station):

    station_df = ma_df[[f'{station}_AQI']]
    fig = px.line(data_frame=station_df,
        x=station_df.index, y=f'{station}_AQI', title = f'{station} AQI Forcasting', template = 'plotly_dark') 

    fig.add_trace(go.Scatter( x=final_forecast.Date, y=final_forecast[f'{station}_Label'].to_numpy(), name="Forecast", line_shape='linear'))
    return fig

app.run_server(mode='inline')

<IPython.core.display.Javascript object>